In [2]:
import subprocess
def call(cmd):
    subprocess.call(cmd , shell=True)

In [3]:
import os
import sys
path = os.path.join(os.path.abspath(os.curdir), '../')
sys.path.append(path)
path = os.path.join(os.path.abspath(os.curdir), '../../')
sys.path.append(path)

In [4]:
import utils
import pandas as pd

# read comic list

In [43]:
def save():
    global comic_list, rubi_list, num_split_list
    _save = lambda obj, path: json.dump(obj, open(path, 'w'))
    _save(comic_list, 'comic_list.json')
    _save(rubi_list, 'rubi_list.json')
    _save(num_split_list, 'num_split_list.json')
    print('saved')
    
def load():
    global comic_list, rubi_list, num_split_list
    _load = lambda path: json.load(open(path, 'r'))
    comic_list = _load('comic_list.json')
    rubi_list = _load('rubi_list.json')
    num_split_list = _load('num_split_list.json')
    print('loaded')

In [44]:
import json
load()
comic_rubi_numsplit_list = list(zip(comic_list, rubi_list, num_split_list))

loaded


In [45]:
comic_rubi_numsplit_list

[('ワンピース', 'onepiece', 100),
 ('キングダム', 'kingdom', 100),
 ('名探偵コナン', 'konan', 100),
 ('ジョジョ', 'jojo', 100),
 ('キン肉マン', 'kiniku-man', 100),
 ('ナルト', 'naruto', 100),
 ('ドラゴンボール', 'doragon-ball', 100),
 ('キャプテン翼', 'kyapten-tsubasa', 50),
 ('ガッシュ', 'gashu', 50),
 ('銀魂', 'gintama', 100),
 ('僕のヒーローアカデミア', 'bokuno-hero-academia', 30),
 ('亜人', 'ajin', 30),
 ('進撃の巨人', 'singeki-no-kyojin', 30),
 ('テラフォーマー', 'terra-former', 30),
 ('ワンパンマン', 'one-punch-man', 30),
 ('約束のネバーランド', 'yakusoku-no-neverland', 30),
 ('Dr.STONE', 'dr-stone', 30),
 ('チェンソーマン', 'tyenso-man', 10),
 ('かぐや様', 'kaguyasama', 20),
 ('ダンジョン飯', 'danjon-meshi', 20),
 ('ワンナウツ', 'oneouts', 10),
 ('BANANA FISH', 'banana-fish', 20),
 ('AKIRA', 'akira', 100),
 ('君に届け', 'kimini-todoke', 5),
 ('狼陛下の花嫁', 'okami-heika', 5),
 ('フィンランドサガ', 'finland-saga', 1),
 ('雀荘のサエコさん', 'saeko', 1),
 ('ジモトがジャパン', 'jimoto', 1),
 ('SILENT NIGHT 翔', 'silent-night-syo', 1),
 ('私立ポセイドン学園高等部', 'poseidon', 1),
 ('チャゲチャ', 'tyagetya', 2),
 ('大泥棒ポルタ', 'poruta', 3)]

# read comic list from db

In [5]:
from comic_crawler.database import init_database, refresh_session, engine, Base, Comic, ComicVolume, init_database
session = refresh_session()

read config → ../config/config.ini


In [6]:
comic_vols_q = session.query(ComicVolume).filter(ComicVolume.published_on.between('2019-06-01', '2019-08-01'))

In [7]:
volume_list = comic_vols_q.all()

2019-08-01 18:27:01,832 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2019-08-01 18:27:01,832 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 18:27:01,840 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2019-08-01 18:27:01,841 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 18:27:01,846 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2019-08-01 18:27:01,847 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 18:27:01,851 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2019-08-01 18:27:01,851 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 18:27:01,854 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2019-08-01 18:27:01,855 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 18:27:01,857 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8mb4) COLLATE utf8mb4_bin AS anon_1
2

In [8]:
len(volume_list)

1568

In [9]:
vol = volume_list[0]

In [ ]:
%%time
comic_list = [session.query(Comic).filter(Comic.id == vol.comic_id).first() for vol in volume_list]

In [11]:
def to_dict(instance):
    return {k:v for k,v in instance.__dict__.items() if '_' != k[0]}

def to_dictlist(instance_list):
    return [to_dict(instance) for instance in instance_list]

In [12]:
df_comic = pd.DataFrame(to_dictlist(comic_list)).rename(columns={'id':'comic_id'})
df_volume = pd.DataFrame(to_dictlist(volume_list)).rename(columns={'id':'volume_id'})

In [13]:
df_volume.shape, df_comic.shape

((1568, 8), (1568, 5))

In [26]:
df_comic_volume = pd.merge(df_comic, df_volume, on='comic_id', how='right')

In [27]:
df_comic_volume = df_comic_volume.drop_duplicates(['comic_id', 'volume_id'])

In [32]:
# df_comic_volume.sort_values('published_on')

In [36]:
po = df_comic_volume.published_on
df_comic_volume_7 = df_comic_volume[('2019-06-30' < po)].sort_values('published_on')

In [86]:
import re
def parse_roma(t):
    t = utils.to_roma(t)# ローマ字に
    t = re.sub(r'[^a-z]', '-', t.lower()).strip() # a-z以外を-に
    t = re.sub(r'[-]+', '-', t) # -の連続をまとめる
    t = re.sub(r'^-|-$', '', t) # 行頭, 行末の-削除
    return t

def extract_roma(roma, n):
    t = roma[:n]
    t = re.sub(r'^-|-$', '', t) # 行頭, 行末の-削除
    return t

In [93]:
df_comic_volume_7['roma'] = df_comic_volume_7.title.apply(lambda title: parse_roma(title))
df_comic_volume_7['roma_10'] = df_comic_volume_7.roma.apply(lambda roma: extract_roma(roma, 10))
df_comic_volume_7['instance_name'] = [f'{row.roma_10}_{row.comic_id}' for i, row in df_comic_volume_7.iterrows()]

In [70]:
a = 'shinsho　orureriannokishihime'

In [73]:
a.strip(' ')

'shinsho\u3000orureriannokishihime'

# define num of instance by comic
漫画ごとのインスタンス数を決める。  
2日で10分ぐらい。1月で2.6時間,   
本当はここも自動化したい。タスクをDBに入れて、それを消化していく形にしたい

In [41]:
from twitterscraper.query import linspace
import datetime as dt
def split_date(begindate, enddate, num_split):
    num_days = (enddate - begindate).days
    dateranges = [begindate + dt.timedelta(days=elem) for elem in linspace(0, num_days, num_split+1)]
    return dateranges

In [42]:
def create_vm_name(basename, dateranges, query=None):
    res_list = [{
        'name': f'{basename}-{since}-{until}',
        'since': since,
        'until': until,
    } for since, until in zip(dateranges[:-1], dateranges[1:])]
    if query is not None:
        for res in res_list:
            res['query'] = query
    return res_list

In [8]:
begindate = dt.date(2006, 3, 21)
enddate = dt.date(2019, 6, 12)
num_split = 10
dateranges = split_date(begindate, enddate, num_split=num_split)

In [9]:
comic_daterange_list = create_vm_name('onepiece', dateranges)
comic_daterange_name_list = [c['name'] for c in comic_daterange_list]

In [10]:
crawl_query_instance_list = create_vm_name('onepiece', dateranges)
for d in crawl_query_instance_list:
    d['query'] = 'ワンピース'

In [11]:
comic_rubi_numsplit_list

[('ワンピース', 'onepiece', 100),
 ('キングダム', 'kingdom', 100),
 ('名探偵コナン', 'konan', 100),
 ('ジョジョ', 'jojo', 100),
 ('キン肉マン', 'kiniku-man', 100),
 ('ナルト', 'naruto', 100),
 ('ドラゴンボール', 'doragon-ball', 100),
 ('キャプテン翼', 'kyapten-tsubasa', 50),
 ('ガッシュ', 'gashu', 50),
 ('銀魂', 'gintama', 100),
 ('僕のヒーローアカデミア', 'bokuno-hero-academia', 30),
 ('亜人', 'ajin', 30),
 ('進撃の巨人', 'singeki-no-kyojin', 30),
 ('テラフォーマー', 'terra-former', 30),
 ('ワンパンマン', 'one-punch-man', 30),
 ('約束のネバーランド', 'yakusoku-no-neverland', 30),
 ('Dr.STONE', 'dr-stone', 30),
 ('チェンソーマン', 'tyenso-man', 10),
 ('かぐや様', 'kaguyasama', 20),
 ('ダンジョン飯', 'danjon-meshi', 20),
 ('ワンナウツ', 'oneouts', 10),
 ('BANANA FISH', 'banana-fish', 20),
 ('AKIRA', 'akira', 100),
 ('君に届け', 'kimini-todoke', 5),
 ('狼陛下の花嫁', 'okami-heika', 5),
 ('フィンランドサガ', 'finland-saga', 1),
 ('雀荘のサエコさん', 'saeko', 1),
 ('ジモトがジャパン', 'jimoto', 1),
 ('SILENT NIGHT 翔', 'silent-night-syo', 1),
 ('私立ポセイドン学園高等部', 'poseidon', 1),
 ('チャゲチャ', 'tyagetya', 2),
 ('大泥棒ポルタ', 'poruta', 3)]

In [365]:
comic_rubi_numsplit_list.sort(key=lambda x: x[2])
comic_rubi_numsplit_list

[('フィンランドサガ', 'finland-saga', 1),
 ('雀荘のサエコさん', 'saeko', 1),
 ('ジモトがジャパン', 'jimoto', 1),
 ('SILENT NIGHT 翔', 'silent-night-syo', 1),
 ('私立ポセイドン学園高等部', 'poseidon', 1),
 ('チャゲチャ', 'tyagetya', 2),
 ('大泥棒ポルタ', 'poruta', 3),
 ('君に届け', 'kimini-todoke', 5),
 ('狼陛下の花嫁', 'okami-heika', 5),
 ('チェンソーマン', 'tyenso-man', 10),
 ('ワンナウツ', 'oneouts', 10),
 ('かぐや様', 'kaguyasama', 20),
 ('ダンジョン飯', 'danjon-meshi', 20),
 ('BANANA FISH', 'banana-fish', 20),
 ('僕のヒーローアカデミア', 'bokuno-hero-academia', 30),
 ('亜人', 'ajin', 30),
 ('進撃の巨人', 'singeki-no-kyojin', 30),
 ('テラフォーマー', 'terra-former', 30),
 ('ワンパンマン', 'one-punch-man', 30),
 ('約束のネバーランド', 'yakusoku-no-neverland', 30),
 ('Dr.STONE', 'dr-stone', 30),
 ('キャプテン翼', 'kyapten-tsubasa', 50),
 ('ガッシュ', 'gashu', 50),
 ('ワンピース', 'onepiece', 100),
 ('キングダム', 'kingdom', 100),
 ('名探偵コナン', 'konan', 100),
 ('ジョジョ', 'jojo', 100),
 ('キン肉マン', 'kiniku-man', 100),
 ('ナルト', 'naruto', 100),
 ('ドラゴンボール', 'doragon-ball', 100),
 ('銀魂', 'gintama', 100),
 ('AKIRA', 'akira', 100)]

In [395]:
begindate = dt.date(2006, 3, 21)
enddate = dt.date.today()
crawl_query_instance_list = []
for comic, rubi, num_split in comic_rubi_numsplit_list:
    dateranges = split_date(begindate, enddate, num_split=num_split)
    crawl_query_instance_list += create_vm_name(rubi, dateranges, comic)

In [416]:
len(crawl_query_instance_list[70:70+51])

51

In [398]:
len(crawl_query_instance_list)

1310

In [417]:
_crawl_query_instance_list = crawl_query_instance_list[70:70+51]

# Create Instance

In [418]:
def make_create_command(vm_index_list, image_name='arb-tweet-crawler-base-3', machine_type='g1-small'):
    vm_name_list = [f'arb-tweet-crawler-{vm_index}' for vm_index in vm_index_list]
    vm_names_text = ' '.join(vm_name_list)
    command = f'gcloud compute  --project "machinelearning-219614"  instances create {vm_names_text} --machine-type "{machine_type}" --zone "us-central1-a" --image {image_name} --image-project "machinelearning-219614"'
    return command

In [419]:
comic_daterange_name_list = [comic['name'] for comic in _crawl_query_instance_list]
create_command = make_create_command(comic_daterange_name_list)

In [420]:
comic_daterange_name_list

['danjon-meshi-2012-11-04-2013-07-04',
 'danjon-meshi-2013-07-04-2014-03-03',
 'danjon-meshi-2014-03-03-2014-10-31',
 'danjon-meshi-2014-10-31-2015-06-30',
 'danjon-meshi-2015-06-30-2016-02-27',
 'danjon-meshi-2016-02-27-2016-10-26',
 'danjon-meshi-2016-10-26-2017-06-25',
 'danjon-meshi-2017-06-25-2018-02-22',
 'danjon-meshi-2018-02-22-2018-10-22',
 'danjon-meshi-2018-10-22-2019-06-22',
 'banana-fish-2006-03-21-2006-11-18',
 'banana-fish-2006-11-18-2007-07-18',
 'banana-fish-2007-07-18-2008-03-16',
 'banana-fish-2008-03-16-2008-11-13',
 'banana-fish-2008-11-13-2009-07-13',
 'banana-fish-2009-07-13-2010-03-12',
 'banana-fish-2010-03-12-2010-11-09',
 'banana-fish-2010-11-09-2011-07-09',
 'banana-fish-2011-07-09-2012-03-07',
 'banana-fish-2012-03-07-2012-11-04',
 'banana-fish-2012-11-04-2013-07-04',
 'banana-fish-2013-07-04-2014-03-03',
 'banana-fish-2014-03-03-2014-10-31',
 'banana-fish-2014-10-31-2015-06-30',
 'banana-fish-2015-06-30-2016-02-27',
 'banana-fish-2016-02-27-2016-10-26',
 '

In [421]:
save_path = './create_vm.sh'
utils.to_shell(create_command, save_path, is_parallel=True)
! cat $save_path

gcloud compute  --project "machinelearning-219614"  instances create arb-tweet-crawler-danjon-meshi-2012-11-04-2013-07-04 arb-tweet-crawler-danjon-meshi-2013-07-04-2014-03-03 arb-tweet-crawler-danjon-meshi-2014-03-03-2014-10-31 arb-tweet-crawler-danjon-meshi-2014-10-31-2015-06-30 arb-tweet-crawler-danjon-meshi-2015-06-30-2016-02-27 arb-tweet-crawler-danjon-meshi-2016-02-27-2016-10-26 arb-tweet-crawler-danjon-meshi-2016-10-26-2017-06-25 arb-tweet-crawler-danjon-meshi-2017-06-25-2018-02-22 arb-tweet-crawler-danjon-meshi-2018-02-22-2018-10-22 arb-tweet-crawler-danjon-meshi-2018-10-22-2019-06-22 arb-tweet-crawler-banana-fish-2006-03-21-2006-11-18 arb-tweet-crawler-banana-fish-2006-11-18-2007-07-18 arb-tweet-crawler-banana-fish-2007-07-18-2008-03-16 arb-tweet-crawler-banana-fish-2008-03-16-2008-11-13 arb-tweet-crawler-banana-fish-2008-11-13-2009-07-13 arb-tweet-crawler-banana-fish-2009-07-13-2010-03-12 arb-tweet-crawler-banana-fish-2010-03-12-2010-11-09 arb-tweet-crawler-banana-fish-2010-11

# Stop Delete Instance

In [329]:
def make_stop_command(vm_index_list):
    vm_name_list = [f'arb-tweet-crawler-{vm_index}' for vm_index in vm_index_list]
    vm_names_text = ' '.join(vm_name_list)
    command = f'gcloud compute  --project "machinelearning-219614"  instances stop {vm_names_text} --async --zone "us-central1-a"'
    return command

def make_delete_command(vm_index_list):
    vm_name_list = [f'arb-tweet-crawler-{vm_index}' for vm_index in vm_index_list]
    vm_names_text = ' '.join(vm_name_list)
    command = f'gcloud compute  --project "machinelearning-219614"  instances delete {vm_names_text} --zone "us-central1-a"'
    return command

In [330]:
c = make_stop_command(vm_index_list)
to_shell(c, './stop_vm.sh')

TypeError: to_shell() missing 1 required positional argument: 'is_parallel'

In [28]:
vm_index_list += [1, 20]

In [29]:
c = make_delete_command(vm_index_list)
to_shell(c, './delete_vm.sh')

In [67]:
# not_run_vm = [i for i in vm_index_list if i not in vm_crawl_index_list]
c = make_delete_command(vm_index_list)
to_shell(c, './delete_vm.sh')
! cat ./delete_vm.sh

gcloud compute  --project "machinelearning-219614"  instances delete arb-tweet-crawler-1 arb-tweet-crawler-2 arb-tweet-crawler-3 arb-tweet-crawler-4 arb-tweet-crawler-5 arb-tweet-crawler-6 arb-tweet-crawler-7 arb-tweet-crawler-8 arb-tweet-crawler-9 arb-tweet-crawler-10 arb-tweet-crawler-11 arb-tweet-crawler-12 arb-tweet-crawler-13 arb-tweet-crawler-14 arb-tweet-crawler-15 arb-tweet-crawler-16 arb-tweet-crawler-17 arb-tweet-crawler-18 arb-tweet-crawler-19 arb-tweet-crawler-20 arb-tweet-crawler-21 arb-tweet-crawler-22 arb-tweet-crawler-23 arb-tweet-crawler-24 arb-tweet-crawler-25 arb-tweet-crawler-26 arb-tweet-crawler-27 arb-tweet-crawler-28 arb-tweet-crawler-29 arb-tweet-crawler-30 --zone "us-central1-a"

# Run crawling at Instance
ローカルからgce内のdockerへ実行コマンドを送るやつ
テストで実行

In [1]:
# container_command = f"python main.py {query}  --since {since} --until {until}"
# docker_command = f"docker run -d -v --rm /home/syunyooo/auto-ranking-blog/crawler/tweet-crawler/:/app tweet-crawler_jupyter {container_command}"
# docker_command = f"git -C /home/syunyooo/auto-ranking-blog/ pull"

In [378]:
def make_command(query, vm_index, since='2006-03-21', until='2019-06-12'):
    user_name = 'syunyooo'
    vm_name = f'arb-tweet-crawler-{vm_index}'
    vm_command = f'sh /home/syunyooo/auto-ranking-blog/crawler/tweet-crawler/scripts/run-docker-crawl-shutdown.sh {query}  --since {since} --until {until}'
    local_command = f'gcloud compute --project "machinelearning-219614" ssh --zone "us-central1-a" "{user_name}@{vm_name}" --command "{vm_command}"'
    return local_command

In [422]:
run_cmd_list = [make_command(c['query'], c['name'], c['since'], c['until']) for c in _crawl_query_instance_list]

In [423]:
len(run_cmd_list)

51

In [ ]:
save_path = './run_crawl.sh'
to_shell(run_cmd_list, save_path, is_parallel=True)
! cat $save_path

# pre

In [38]:
comic_dict

{'hall_of_frame_serialized': ['ワンピース', 'キングダム', '名探偵コナン', 'ジョジョ', 'キン肉マン'],
 'hall_of_frame_ended': ['ナルト', 'ドラゴンボール', 'キャプテン翼', 'ガッシュ', '銀魂'],
 'famous_from_before': ['僕のヒーローアカデミア', '亜人', '進撃の巨人', 'テラフォーマー', 'ワンパンマン'],
 'famous_recently': ['約束のネバーランド', 'Dr.STONE', 'チェンソーマン', 'かぐや様', 'ダンジョン飯'],
 'geek': ['ワンナウツ', 'BANANA FISH', 'AKIRA', '君に届け', '狼陛下の花嫁'],
 'obscure': ['フィンランドサガ',
  '雀荘のサエコさん',
  'ジモトがジャパン',
  'SILENT NIGHT 翔',
  '私立ポセイドン学園高等部',
  'チャゲチャ',
  '大泥棒ポルタ']}

In [40]:
comic_list = []
for k,v in comic_dict.items():
    comic_list += v

In [43]:
# 処理済み
ignore_comic_list = ['ワンピース', '進撃の巨人', 'チェンソーマン', 'ジョジョ', 'ジモトがジャパン',]

In [44]:
comic_list = [comic for comic in comic_list if comic not in ignore_comic_list]